<a href="https://colab.research.google.com/github/sp8rks/MaterialsInformatics/blob/main/worked_examples/agents/langGraph_material_selection_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 LangGraph Tutorial: Multi-Criteria Materials Selector

Welcome to this beginner-friendly tutorial on **LangGraph**! 🎉

In this notebook, we'll walk through how to use LangGraph to create a multi-agent system that selects materials for a high-temperature turbine blade. You'll learn:
- What nodes (aka agents) are and how they work
- How to define a shared state
- How to design parallel workflows
- How to compile and run a graph of language model agents

This is perfect for new students exploring **agents, LLM workflows**, and **materials informatics**.

Let's get started! 🔧


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

key_path = '/content/drive/My Drive/teaching/5540-6640 Materials Informatics/openAI_Apikey.txt'
with open(key_path, 'r') as f:
    os.environ["OPENAI_API_KEY"] = f.read().strip()


Mounted at /content/drive


## 📋 Step 3: Define the Shared State

This defines the structure of the data shared between nodes. Each node reads from and writes to this state.


## 🤖 Step 4: Setup the Language Model

We'll use `gpt-3.5-turbo` with a low temperature for deterministic answers.
If you want a better model, you could try `gpt-4o` with example code below

`llm = ChatOpenAI(model="gpt-4o", temperature=0.2)`



## 🧱 Step 5: Define Each Node (Agent)

Each node performs one task in the pipeline. We define them as Python functions that use LLMs to process input and return part of the state.


## 🧩 Step 6: Build the LangGraph

Here we create the graph, add the nodes, and define how they connect.


## 🚀 Step 7: Run the Graph

We provide an initial query, invoke the graph, and print the final recommendation.


In [ ]:
# LangGraph-Based Multi-Criteria Materials Selector
# Install dependencies
!pip install -q langgraph langchain langchain_openai langchain_community
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from typing import TypedDict, List, Optional, Dict, Any

# --- Shared State Definition ---
class MaterialState(TypedDict):
    query: str
    materials_found: Optional[List[str]]
    properties: Optional[str]
    cost_estimate: Optional[str]
    sustainability: Optional[str]
    comparison: Optional[str]
    reasoning: Optional[str]
    recommendation: Optional[str]

# --- Language Model Setup ---
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)
llm = ChatOpenAI(
    model="gpt-4o", temperature=0.2,
    system_message="You are an expert materials science advisor who always supports answers with clear evidence."
)


# --- Node 1: Material Finder ---
finder_prompt = PromptTemplate.from_template(
    """
    You are a materials scientist. Find 3–5 materials suitable for this requirement:
    "{query}"
    Return a list of materials.
    """
)
finder_chain = LLMChain(llm=llm, prompt=finder_prompt)

def material_finder_node(state: MaterialState) -> Dict[str, Any]:
    query = state["query"]
    response = finder_chain.run(query=query)
    materials = [line.strip("- ").strip() for line in response.strip().split("\n") if line.strip()]
    return {"materials_found": materials}

# --- Node 2a: Property Extractor ---
property_prompt = PromptTemplate.from_template(
    """
    List key physical or chemical properties for each of the following materials:
    {materials}
    """
)
property_chain = LLMChain(llm=llm, prompt=property_prompt)

def property_node(state: MaterialState) -> Dict[str, Any]:
    materials = "\n".join(state["materials_found"])
    properties = property_chain.run(materials=materials)
    return {"properties": properties}

# --- Node 2b: Cost Estimator ---
cost_prompt = PromptTemplate.from_template(
    """
    Estimate the cost per kilogram (USD) for each of the following materials:
    {materials}
    Format as: - Material: $X/kg
    """
)
cost_chain = LLMChain(llm=llm, prompt=cost_prompt)

def cost_node(state: MaterialState) -> Dict[str, Any]:
    materials = "\n".join(state["materials_found"])
    cost_estimate = cost_chain.run(materials=materials)
    return {"cost_estimate": cost_estimate}

# --- Node 2c: Sustainability Evaluator ---
sustain_prompt = PromptTemplate.from_template(
    """
    Evaluate the sustainability of these materials.
    Consider recyclability, environmental extraction impact, and CO2 footprint:
    {materials}
    """
)
sustain_chain = LLMChain(llm=llm, prompt=sustain_prompt)

def sustainability_node(state: MaterialState) -> Dict[str, Any]:
    materials = "\n".join(state["materials_found"])
    sustainability = sustain_chain.run(materials=materials)
    return {"sustainability": sustainability}

# --- Node 3: Comparator ---
compare_prompt = PromptTemplate.from_template(
    """
    Compare the following materials based on this query: "{query}"
    PROPERTIES:
    {properties}
    COST:
    {cost_estimate}
    SUSTAINABILITY:
    {sustainability}
    Give a balanced trade-off analysis.
    """
)
compare_chain = LLMChain(llm=llm, prompt=compare_prompt)

# Added a check function to make sure all required data is available
def should_compare(state: MaterialState) -> bool:
    return (state.get("properties") is not None and
            state.get("cost_estimate") is not None and
            state.get("sustainability") is not None)

def comparator_node(state: MaterialState) -> Dict[str, Any]:
    # Only run if we have all required data
    if not should_compare(state):
        return {}

    comparison = compare_chain.run(
        query=state["query"],
        properties=state["properties"],
        cost_estimate=state["cost_estimate"],
        sustainability=state["sustainability"]
    )
    return {"comparison": comparison}

# --- Node 4: Reasoning ---
reasoning_prompt = PromptTemplate.from_template(
    """
    Based on this comparison:
    {comparison}
    What are the major trade-offs and constraints?
    Summarize your reasoning.
    """
)
reasoning_chain = LLMChain(llm=llm, prompt=reasoning_prompt)

def reasoning_node(state: MaterialState) -> Dict[str, Any]:
    reasoning = reasoning_chain.run(
        comparison=state["comparison"]
    )
    return {"reasoning": reasoning}

# --- Node 5: Final Recommender ---
recommend_prompt = PromptTemplate.from_template(
    """
    Based on the reasoning and comparison, recommend the best material.
    Materials: {materials}
    Comparison: {comparison}
    Reasoning: {reasoning}
    """
)
recommend_chain = LLMChain(llm=llm, prompt=recommend_prompt)

def recommend_node(state: MaterialState) -> Dict[str, Any]:
    recommendation = recommend_chain.run(
        materials="\n".join(state["materials_found"]),
        comparison=state["comparison"],
        reasoning=state["reasoning"]
    )
    return {"recommendation": recommendation}

# --- Graph Construction ---
graph_builder = StateGraph(MaterialState)
graph_builder.add_node("finder", material_finder_node)
graph_builder.add_node("get_properties", property_node)
graph_builder.add_node("get_cost", cost_node)
graph_builder.add_node("get_sustain", sustainability_node)
graph_builder.add_node("compare", comparator_node)
graph_builder.add_node("reason", reasoning_node)
graph_builder.add_node("recommend", recommend_node)

# Set entry point
graph_builder.set_entry_point("finder")

# Fan-out from finder
graph_builder.add_edge("finder", "get_properties")
graph_builder.add_edge("finder", "get_cost")
graph_builder.add_edge("finder", "get_sustain")

# Use conditional edges to join the parallel branches
# Each parallel task leads to compare when complete
graph_builder.add_edge("get_properties", "compare")
graph_builder.add_edge("get_cost", "compare")
graph_builder.add_edge("get_sustain", "compare")

# Use a conditional edge from compare to reason to ensure all data is ready
def check_compare_ready(state: MaterialState) -> str:
    if state.get("comparison") is not None:
        return "reason"
    else:
        return "compare"  # Loop back until data is ready

graph_builder.add_conditional_edges(
    "compare",
    check_compare_ready
)

# Final flow
graph_builder.add_edge("reason", "recommend")
graph_builder.add_edge("recommend", END)

# Compile graph
graph = graph_builder.compile()

# --- Run Example ---
initial_state = {
    "query": "We need a material for a turbine blade operating at 1250°C under high stress."
}
result = graph.invoke(initial_state)

from pprint import pprint
pprint(result)
print("\n\n✅ Final Recommendation:\n")
print(result["recommendation"])

{'comparison': 'Based on the properties, cost, and sustainability factors '
               'provided, a balanced trade-off analysis can be made as '
               'follows:\n'
               '\n'
               '1. Nickel-based superalloys:\n'
               '- Properties: Nickel-based superalloys have high temperature '
               'strength, corrosion resistance, oxidation resistance, creep '
               'resistance, and high thermal conductivity, making them '
               'suitable for turbine blades operating at 1250°C under high '
               'stress.\n'
               '- Cost: Nickel-based superalloys are relatively affordable at '
               '$50/kg.\n'
               '- Sustainability: While nickel-based superalloys are '
               'recyclable, the process can be complex and costly. The '
               'extraction of nickel can have negative environmental impacts, '
               'and the production process can have a high CO2 footprint.\n'
             